In [173]:
# IPython Imports
# Used to make notebook wider, comment out for normal notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import clear_output

In [174]:
# Import Libraries
import boto3
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from typing import List, Dict, Callable

In [175]:
# GLOBAL HELPER FUNCTIONS
def print_ref(d_obj:Dict[str, any], row_len:int=4) -> None:
    ref = \
f""" ---------------------------
| OBJECT KEYS FOR REFERENCE |
 ---------------------------
"""
    row = 0
    for key in d_obj.keys():
        if row == row_len: 
            row = 0
            ref += "\n"
        ref += f" {key} |"
        row += 1
    print(ref)
    
def parse_col(col_list:List[str], data_frame:pd.DataFrame=None) -> List[str]:
    res, len_ = [], len(col_list) + 1
    df_exists = data_frame is not None
    for num, col in enumerate(col_list):
        example = data_frame[col].iloc[0] if df_exists else "None"
        ans = input(f"""
Item #: {num + 1}/{len_}
Column: {col}
Example: {example}
[y/n] Default[n] -> """).lower()
        clear_output()
        if ans == "y":
            res.append(col)
    return res


In [176]:
# GLOBAL VARS
BKT_NAME = 'ds-data-2020'

# DATASET NAMES
the_cc = "thecarconnectiondataset.csv" # Main Dataset
motortrend = "motortrend1974.csv" # Currently Not Using
car_feat_msrp = "carfeaturesmsrp.csv" # Currently Not Using

# SET DATA PATH/KEYS
KEY = the_cc

In [177]:
# Create boto3 s3 Client Object
s3 = boto3.client('s3')
# Create s3 Bucket Object -> Dict
obj = s3.get_object(Bucket=BKT_NAME, Key=KEY)

# obj_keys = obj.keys()
print_ref(obj)


 ---------------------------
| OBJECT KEYS FOR REFERENCE |
 ---------------------------
 ResponseMetadata | AcceptRanges | LastModified | ContentLength |
 ETag | ContentType | Metadata | Body |


In [178]:
# Might have to chunk data
chk_size = 2000


# Convert Bucket File to transposed pd.DataFrame
auto_df = pd.read_csv(obj['Body'], index_col=0, header=None, low_memory=False).T

In [179]:
# Rename known NaN columns
auto_df.rename(columns={ auto_df.columns[0]: "Model" }, inplace = True)

In [180]:
# Get only wanted columns
# col = parse_col(auto_df.columns, auto_df)

# col = ["Model", 'MSRP', 'Gas Mileage', 'Engine', 'EPA Class', 'Style Name', 'Drivetrain', 'Passenger Capacity', 'Passenger Doors', 'Body Style', 'Transmission', 'Base Curb Weight (lbs)', 'Wheelbase (in)', 'Min Ground Clearance (in)', 'Track Width, Front (in)', 'Track Width, Rear (in)', 'Height, Overall (in)', 'Fuel Economy Est-Combined (MPG)', 'SAE Net Torque @ RPM', 'Fuel System', 'Engine Type', 'SAE Net Horsepower @ RPM', 'Displacement', 'First Gear Ratio (:1)', 'Sixth Gear Ratio (:1)', 'Fourth Gear Ratio (:1)', 'Seventh Gear Ratio (:1)', 'Second Gear Ratio (:1)', 'Reverse Ratio (:1)', 'Fifth Gear Ratio (:1)', 'Eighth Gear Ratio (:1)', 'Third Gear Ratio (:1)', 'Final Drive Axle Ratio (:1)', 'Steering Type', 'Front Tire Size', 'Rear Tire Size']

In [181]:
# Save wanted col to txt file
# with open("data/wanted_col.txt", "w") as text_file:
#     for item in col:
#         text_file.write(item + "\n")

In [182]:
# Get list of column from wanted_col.txt
wanted_col = open("data/wanted_col.txt", "r").read().splitlines()
# Extract columns to new df
auto_df_clean = auto_df[wanted_col].copy()

In [183]:
# Clean Model Column to only contain year and name
auto_df_clean["Model"] =  [i.partition('Specs')[0] for i in auto_df_clean.Model]

In [184]:
# Create Year Column from Model Column
auto_df_clean.insert(0, "Year", [re.search("([^\s]+)",x).group() for x in auto_df_clean["Model"]])

In [185]:
# Cast Year Column to Int
auto_df_clean["Year"] = auto_df_clean["Year"].astype(int) 

In [186]:
# Clean Model Column to only contain name
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group()) for i in auto_df_clean.Model]

In [187]:
# Create Brand Column from Model Column
auto_df_clean.insert(1, "Brand", [re.search("([^\s]+)", x).group() for x in auto_df_clean["Model"]])

In [188]:
# Clean Model Column to only contain model
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group(1)) for i in auto_df_clean.Model]

In [189]:
# Get rid of all nan values in Gas Mileage
nans = pd.notnull(auto_df_clean['Gas Mileage'])
auto_df_clean = auto_df_clean[nans]

In [190]:
# Split Gas Mileage into City and Hwy
auto_df_clean.insert(4, "Gas Mileage (City)", [(x.partition("/")[0].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])
auto_df_clean.insert(5, "Gas Mileage (Hwy)", [(x.partition("/")[2].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])

In [191]:
# Gas Mileage to INT
auto_df_clean['Gas Mileage (City)'] = pd.to_numeric(auto_df_clean['Gas Mileage (City)'])
auto_df_clean['Gas Mileage (Hwy)'] = pd.to_numeric(auto_df_clean['Gas Mileage (Hwy)'])

In [192]:
# Drop Gas Mileage Column after split
auto_df_clean.drop(columns=["Gas Mileage"], inplace=True)

In [193]:
# Clean Gear Ratio Columns
gears = ["First Gear Ratio (:1)", "Second Gear Ratio (:1)", "Third Gear Ratio (:1)", "Fourth Gear Ratio (:1)", "Fifth Gear Ratio (:1)", "Sixth Gear Ratio (:1)", "Seventh Gear Ratio (:1)", "Eighth Gear Ratio (:1)", "Reverse Ratio (:1)", "Final Drive Axle Ratio (:1)"]

# Reorder Gears Numerically
for idx, column in enumerate(gears, 26):
    col = auto_df_clean[column]
    auto_df_clean.drop(labels=[column], axis=1,inplace = True)
    auto_df_clean.insert(idx, column, col)
    
# Cast NaN to values in col with "- TBD -"
for gear in gears:
    auto_df_clean.loc[auto_df_clean[gear].str.contains("TBD", na=False), gear] = np.nan

In [194]:
# Cast float to values in col with string arithmetic
"Notes: Some gear columns need to be sep with (,), (/), or (" ") delimiters"
for gear in gears:
    auto_df_clean[gear] = pd.to_numeric([
        (x.partition(",")[0]) if ((type(x) == str) and ("," in x)) 
        else (x.partition("/")[0]) if ((type(x) == str) and ("/" in x))
        else (x.partition("-")[0]) if ((type(x) == str) and ("-" in x))
        else (x.partition(" ")[0]) if ((type(x) == str) and (" " in x))
        else x for x in auto_df_clean[gear]
    ], downcast="float", errors="coerce")

In [195]:
# Cast MSRP to Numeric Float
auto_df_clean["MSRP"] = auto_df_clean["MSRP"].replace('[\$,]', '', regex=True).astype(float)
# Rename MSRP to indicate $
auto_df_clean.rename(columns={ "MSRP": "MSRP($)" }, inplace = True)

In [196]:
# Get rid of all nan values in Horsepower/Torque
nans = pd.notnull(auto_df_clean['SAE Net Torque @ RPM'])
auto_df_clean = auto_df_clean[nans]

In [197]:
# Split SAE Horsepower into Horsepower and @ RPM
auto_df_clean.insert(4, "Horsepower", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])
auto_df_clean.insert(5, "@ RPM (HP)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])

In [198]:
# Split SAE Torque into Torque and @ RPM
auto_df_clean.insert(6, "Torque", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Torque @ RPM"]])
auto_df_clean.insert(7, "@ RPM (TQ)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Torque @ RPM"]])

In [199]:
# Clean and Cast Power Values (Horsepower / Torque)
pow_rpm_ratings = ["Horsepower", "Torque", "@ RPM (HP)", "@ RPM (TQ)"]
"Notes: Some Power columns need to be sep with (@), or (-) delimiters"
# Cast NaN to values in col with empty ratings or "- TBD -"
for col in pow_rpm_ratings:
    auto_df_clean.loc[auto_df_clean[col].str.contains("TBD", na=False), col] = np.nan
    if col in pow_rpm_ratings[:2]:
        auto_df_clean[col] = pd.to_numeric(auto_df_clean[col], downcast="float", errors="coerce")

In [200]:
# Drop SAE Horsepower and SAE Torque Columns after splits
auto_df_clean.drop(columns=["SAE Net Horsepower @ RPM"], inplace=True)
auto_df_clean.drop(columns=["SAE Net Torque @ RPM"], inplace=True)

In [245]:
auto_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26284 entries, 1 to 32316
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             26284 non-null  int64  
 1   Brand                            26284 non-null  object 
 2   Model                            26284 non-null  object 
 3   MSRP($)                          26261 non-null  float64
 4   Horsepower                       26250 non-null  float32
 5   @ RPM (HP)                       26207 non-null  object 
 6   Torque                           26242 non-null  float32
 7   @ RPM (TQ)                       26238 non-null  object 
 8   Gas Mileage (City)               26284 non-null  int64  
 9   Gas Mileage (Hwy)                26276 non-null  float64
 10  Engine                           26283 non-null  object 
 11  EPA Class                        26283 non-null  object 
 12  Style Name        

In [202]:
# auto_df_clean.head()

In [208]:
# auto_df_clean.describe()

In [238]:
high_hp_brands = auto_df_clean[["Year", "Brand", "Model", "Style Name", "Horsepower", "Torque"]][auto_df_clean["Horsepower"] > 600].groupby("Brand")

In [251]:
# auto_df_clean[auto_df_clean["Brand"] == "Porsche"]

In [256]:
# auto_df_clean["Engine Type"].unique()

In [242]:
auto_df_clean["Brand"].unique()

array(['Acura', 'Alfa', 'Aston', 'Audi', 'Bentley', 'BMW', 'Buick',
       'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'Ferrari', 'FIAT',
       'Ford', 'Genesis', 'GMC', 'Honda', 'Hyundai', 'INFINITI', 'Jaguar',
       'Jeep', 'Kia', 'Lamborghini', 'Land', 'Lexus', 'Lincoln', 'Lotus',
       'Maserati', 'Mazda', 'McLaren', 'Mercedes-Benz', 'MINI',
       'Mitsubishi', 'Nissan', 'Porsche', 'Ram', 'Rolls-Royce', 'smart',
       'Subaru', 'Toyota', 'Volkswagen', 'Volvo'], dtype=object)